In [2]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [4]:
# get url from amazon
url="https://www.amazon.in/s?k=laptop&crid=2WIJCJ2VJ9WLU&sprefix=laptop%2Caps%2C349&ref=nb_sb_noss_2"

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/145.0.0.0 Safari/537.36 Edg/145.0.0.0"
}

In [6]:
# send a get request to the url
response=requests.get(url,headers=headers)
response

<Response [200]>

In [7]:
# scrap data from the website
for page in range(1,3):
    params={'k':'laptop','page':page}
    response=requests.get(url,headers=headers,params=params)
    

In [8]:
response

<Response [200]>

In [9]:
# beautifulsoup to parse the html content
soup=BeautifulSoup(response.content,"html.parser")


In [10]:
# products find div class name : class="sg-col-inner"
products=soup.find_all("div",{'data-component-type':'s-search-result'})
print(len(products))

22


In [11]:
#create empty lists to store the data
product_names=[]
# loop through the products and extract the names
for product in products:
    #find the product name
    title_tag=product.h2
    if title_tag:
        product_name=title_tag.text.strip()
        
    price=product.find("span",class_="a-price-whole")
    if price:
        product_price=price.text.strip()
        # product_names.append(product_price)
    match=re.match(r'^\W*([A-Za-z]+)',product_name)
    brand=match.group(1).upper() if match else "UNKNOWN"
    # Extract ram from the product 
    ram_match=re.search(r'(\d+GB RAM)',product_name)
    ram=ram_match.group(1) if ram_match else "No RAM info"
    #Extract Rating from the product
    rating_tag=product.find("span",class_="a-icon-alt")
    if rating_tag:
        rating=rating_tag.text.strip()
        #Extract only the numeric part of the rating
        rating_match=re.match(r'(\d+(\.\d+)?) out of 5 stars',rating)
        rating=rating_match.group(1) if rating_match else "No rating"
    else:
        rating="No rating"
    # extract the storage information from the product name
    storage_match=re.search(r'(\d+GB\s*(SSD|HDD))',product_name)
    storage=storage_match.group(1) if storage_match else "No Storage info"
    #Extract the color information from the product name
    color_match=re.search(r'(Black|White|Silver|Blue|Red)',product_name)
    color=color_match.group(1) if color_match else "No Color info"
    # Extract the windows information from the product name
    windows_match=re.search(r'(Windows 10|Windows 11)',product_name)
    windows=windows_match.group(1) if windows_match else "No Windows info"

    product_names.append({
        'Title':product_name,
        'Price':product_price,
        'Brand':brand,
        'RAM':ram,
        'Rating':rating,
        'Storage':storage,
        'Color':color,
        'windows':windows
    })
    

In [12]:
for i in product_names:
    print("Title:", i["Title"])
    print("Price:",i["Price"])
    print("Brand:",i["Brand"])
    print("RAM:",i["RAM"])
    print("Rating:",i["Rating"])
    print("Storage:",i["Storage"])
    print("Color:",i["Color"])
    print("Windows:",i["windows"])
    print("-"*50)

Title: Dell 15, Intel Core i5 13th Gen-1334U Processor, 16GB DDR4, 512GB SSD, FHD, 15.6"/39.62cm, Windows 11, MSO 2024, Grey, 1.66kg, [Vostro 3530], 120Hz 250 nits, 15 Month McAfee, Thin & Light, Laptop
Price: 56,990
Brand: DELL
RAM: No RAM info
Rating: 3.7
Storage: 512GB SSD
Color: No Color info
Windows: Windows 11
--------------------------------------------------
Title: Dell 15, AMD Ryzen 7-7730U, 16GB DDR4, 512GB SSD, FHD, 15.6"/39.6cm, Windows 11, MSO-24, Platinum Silver, 1.63Kg, [Dell 15], 120Hz 250nits, AMD Radeon Graphics, Standard Keyboard, Thin & Light, Laptop
Price: 51,990
Brand: DELL
RAM: No RAM info
Rating: 3.5
Storage: 512GB SSD
Color: Silver
Windows: Windows 11
--------------------------------------------------
Title: ULTIMUS APEX Laptop Intel Pentium Quad Core 8 GB DDR3 512 GB SSD Expandable~1TB 14.1 HD IPS Laptop Anti-Glare Ultra Thin Bezel 180° Hinge 3.0x3 USB HDMI SD Card Slot Win 11 Home 1.25KG Silver
Price: 18,990
Brand: ULTIMUS
RAM: No RAM info
Rating: 3.4
Storage

In [13]:
len(product_names)

22

In [14]:
df=pd.DataFrame(product_names)
df

,Title,Price,Brand,RAM,Rating,Storage,Color,windows
0,"Dell 15, Intel Core i5 13th Gen-1334U Processo...","56,990",DELL,No RAM info,3.7,512GB SSD,No Color info,Windows 11
1,"Dell 15, AMD Ryzen 7-7730U, 16GB DDR4, 512GB S...","51,990",DELL,No RAM info,3.5,512GB SSD,Silver,Windows 11
2,ULTIMUS APEX Laptop Intel Pentium Quad Core 8 ...,"18,990",ULTIMUS,No RAM info,3.4,No Storage info,Silver,No Windows info
3,"acer Aspire Go 14,14th Gen, Intel Core Ultra 5...","53,326",ACER,No RAM info,3.7,No Storage info,No Color info,No Windows info
4,"HP Smartchoice Omen AMD Ryzen 7 7840Hs, 8GB RT...","1,26,490",HP,No RAM info,4.0,No Storage info,Black,No Windows info
5,"Lenovo V14 Intel Core i5 13th Gen 14"" FHD (192...","53,760",LENOVO,16GB RAM,3.8,512GB SSD,No Color info,Windows 11
6,"ASUS Gaming V16 (2025), 14th Gen,Intel Core 7 ...","99,990",ASUS,16GB RAM,5.0,512GB SSD,Black,Windows 11
7,"STRIFF Laptop Tabletop Stand, Adjustable Lapto...",199,STRIFF,No RAM info,4.2,No Storage info,Black,No Windows info
8,"HP 15 255R G10, AMD Ryzen 5 7535U Hexa Core (6...","38,990",HP,No RAM info,4.6,No Storage info,Silver,Windows 11
9,Thomson NEO 14.1 Inch IN-P14C Intel Celeron Du...,"12,990",THOMSON,No RAM info,2.7,No Storage info,Silver,No Windows info
